#### This project is created to  focus on understanding and learning about pipelines and its working , rather than creating a best model.


#### This is the hassle way, without using transformers and all, in file 03.ipynb its shown , how to do it the easy and good way.This is for context.

In [1]:
import pandas as pd
import numpy as np

#### Importing Dataset

In [2]:
df= pd.read_csv('train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

#### Removing the columns that we dont need now

In [6]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [7]:
X=df.drop(columns='Survived')
y=df['Survived']

#### Train Test split

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=42)

In [9]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


#### Applying Imputation to handle missing values. we need to apply imputation to Age column and embarked columns,since those column contain missiing values

In [10]:
# Applying imputation
# Will fill all the missing values 

from sklearn.impute import SimpleImputer

si_age = SimpleImputer() #### age-> with mean
si_embarked = SimpleImputer(strategy='most_frequent')  #### embarked-> witht the most frequently occuring value


#### Impute and Trasnfrom age and embark columns
X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_embarked = si_embarked.fit_transform(X_train[['Embarked']])

X_test_age = si_age.transform(X_test[['Age']])
X_test_embarked = si_embarked.transform(X_test[['Embarked']])

#### Apply OneHotEncoding nominal- categorical column. In this case,we apply OneHotEncoding to  Sex and Embarked Column

In [11]:
from sklearn.preprocessing import OneHotEncoder

####Encode the Sex and Embarked column.
ohe_sex = OneHotEncoder(sparse_output=False,handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse_output=False,handle_unknown='ignore')

####fit and transfrom the Sex and Embarked column. 
X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

X_test_sex = ohe_sex.transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.transform(X_test_embarked)

In [12]:
X_train_sex

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [13]:
X_train_embarked

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

#### so now we have three new arrays
#### for age, sex and embarked
#### X_train_age,X_train_sex,X_train_embarked

#### since we have new arrays for the age,sex and embarked column, we will remove the age, sex, embarked column, that we had and replace those with new ie X_train_age,X_train_sex,X_train_embarked

#### Drop the columns in train  and testing set

In [14]:
X_train_rem = X_train.drop(columns=['Sex','Age','Embarked']) 

In [15]:
X_test_rem = X_test.drop(columns=['Sex','Age','Embarked'])

#### Concatenate the training and testing dataset with new columns

In [16]:
X_train_transformed = np.concatenate((X_train_rem,X_train_age,X_train_sex,X_train_embarked),axis=1)
X_test_transformed = np.concatenate((X_test_rem,X_test_age,X_test_sex,X_test_embarked),axis=1)

In [17]:
X_train_transformed

array([[1., 0., 0., ..., 0., 0., 1.],
       [2., 0., 0., ..., 0., 0., 1.],
       [3., 0., 0., ..., 0., 0., 1.],
       ...,
       [3., 2., 0., ..., 0., 0., 1.],
       [1., 1., 2., ..., 0., 0., 1.],
       [1., 0., 1., ..., 0., 0., 1.]])

In [18]:
X_train_transformed.shape

(712, 10)

In [19]:
X_test_transformed.shape

(179, 10)

#### Using DecisionTreeClassifier to train the model

In [20]:
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier()
clf.fit(X_train_transformed,y_train)

DecisionTreeClassifier()

#### Predicting with testing dataset

In [21]:
y_pred= clf.predict(X_test_transformed)

#### Checking the accuracy

In [22]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_pred)

In [23]:
print(accuracy)

0.7821229050279329


#### Exporting the model for future use case.

In [26]:
import pickle

In [27]:
pickle.dump(ohe_sex,open('models/ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('models/ohe_embarked.pkl','wb'))
pickle.dump(clf,open('models/clf.pkl','wb'))